# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and Pandas skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and save the information in a csv
* Perform pagination to retrieve troves of data!
* Write Pandas code to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the data questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Plan out what . One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Save the data you pull as a csv

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Save the data you pull as a csv

6. Take all of the business IDs from your business search, and  using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and save them in a csv.

7. Write Pandas code to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your files.


 
## Data Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to think about what tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your Pandas Dataframes. 


https://www.yelp.com/developers/documentation/v3/get_started

In [2]:
import requests
import json
import pandas as pd
import numpy as np

## Part 2 - Create ETL pipeline for the business data from the API
ETL = EXTRACT TRANSFORM LOAD

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [3]:
#Client_ID and API_Key
client_id = '9oud9pgPIhoXYt0000Ulyg'
api_key = 'LK2ZoCL5UXpQUnH6ZjkgLU8nYy0xAeIFvKqam8DfkZZK_jHIPGttsHpEXhZyqQKVGuAFkqhHdAZ7Cj7yINRVhvxu08tizaxZfR5jrnGbRTwNTE5WqQPGjifRLpmPXnYx'

In [4]:
#Business Endpoint URL and Review Endpoint URL

url =  'https://api.yelp.com/v3/businesses/search'
rurl = 'https://api.yelp.com/v3/businesses/{id}/reviews'

In [5]:
#Parameters of URL_Params

term = 'Japanese Restaurants'
location = 'Monroe'
categories = ['susi', 'japanese']

In [6]:
url_params = {  'term' : term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'categories' : categories,
                'limit' : 50}

In [165]:
# Write a function to make a call to the yelp API

In [7]:
def yelp_call(url_params, api_key):
    url =  'https://api.yelp.com/v3/businesses/search'
    
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()
#     print(response.status_code)
    return data

In [10]:
called_data = yelp_call(url_params, api_key)

In [11]:
called_data.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
called_data['businesses'][0]

{'id': '5haC5qvmyOzMDo2x9CVw2Q',
 'alias': 'akane-woodinville',
 'name': 'Akane',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/zKMUXCcKSj_BEDSyyJ51cw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/akane-woodinville?adjust_creative=9oud9pgPIhoXYt0000Ulyg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=9oud9pgPIhoXYt0000Ulyg',
 'review_count': 245,
 'categories': [{'alias': 'ramen', 'title': 'Ramen'},
  {'alias': 'noodles', 'title': 'Noodles'}],
 'rating': 4.0,
 'coordinates': {'latitude': 47.75561, 'longitude': -122.15139},
 'transactions': ['delivery', 'pickup'],
 'price': '$$',
 'location': {'address1': '14136 NE Woodinville Duvall Rd',
  'address2': '',
  'address3': None,
  'city': 'Woodinville',
  'zip_code': '98072',
  'country': 'US',
  'state': 'WA',
  'display_address': ['14136 NE Woodinville Duvall Rd',
   'Woodinville, WA 98072']},
 'phone': '+14252862243',
 'display_phone': '(425) 286-2243',
 'distance': 17230.898259053487}

In [13]:
called_data['total']

38

In [171]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

In [172]:
type(called_data.keys())

dict_keys

In [16]:
def parse_results(results):
    parsed_data = []
    
    for item in results:
        if 'price' in item.keys():
            biz_list = [item['id'],
                        item['name'],
                        item['review_count'],
                        item['rating'],
                        item['price']]
        else:
            biz_list = [item['id'],
                        item['name'],
                        item['review_count'],
                        item['rating'],
                        np.nan]
            
        parsed_data.append(biz_list)
   
    return parsed_data

In [222]:
# first = parse_results(called_data['businesses'])
#first

In [175]:
#function to save parsed results into a df and overwrite it to a csv file

In [17]:
columns = ['ID', 'Name', 'Review_Count', 'Rating', 'Price_Rank']
nothing = pd.DataFrame(columns=columns)
nothing.to_csv('tattoo_api.csv')

In [18]:
nothing

,ID,Name,Review_Count,Rating,Price_Rank


In [19]:
def data_save(parsed_results, csv_filename):
    
    columns = ['ID', 'Name', 'Review_Count', 'Rating', 'Price_Rank']
    
    update = pd.read_csv(csv_filename, index_col=0)
    
    df1 = pd.DataFrame(parsed_results, columns=columns)
    
    df_list = [update, df1]
    
    ultra = pd.concat(df_list)
    
    return ultra.to_csv(csv_filename) 


In [20]:
# Write a script that combines the three functions above into a single process.

# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < 477:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results['businesses'])
    
    # use your function to insert your parsed results into the db
    data_save(parsed_results, 'tattoo_api.csv')
    #increment the counter by 50 to move on to the next results
    cur += 50

In [21]:
tattoo_df = pd.read_csv('tattoo_api.csv', index_col=0)

In [181]:
#tattoo_df.set_index('ID', inplace=True)

In [29]:
tattoo_df

,ID,Name,Review_Count,Rating,Price_Rank
0,5haC5qvmyOzMDo2x9CVw2Q,Akane,245,4.0,$$
1,zSndGritrDoms1Mr5a4YcQ,Sora Sushi Restaurant,150,4.0,$$
2,KeSd1T3rPx0C4ZJ5aXLskQ,Tokyo Japanese Steak House,142,3.5,$$
3,7MJz76IFbCLfWDixFb6BmQ,Manse Teriyaki,49,4.0,$
4,8ikFTiDK-gHvjl2qroQJPw,Kami Teriyaki,25,3.0,$
5,LbtPLHnzPrHrBz1SMrjSuQ,Best Teriyaki & Sushi,84,4.0,$$
6,jEO9UIK46D-YAtSDY-1J8w,China 1,27,2.5,$$
7,BUQQF2QUuZXa6E3q3KzBxw,Twins Asian Bistro,31,4.5,NaN
8,fyYtjBwks_Ts27ZUWrzczw,Sushi Connections,237,3.0,$$
9,qn_xkreIKL5RrdypG1yArg,Teriyaki Bowl,97,4.5,$


In [27]:
condition = tattoo_df['Rating'] >= 4.5

In [28]:
tattoo_df[condition]

,ID,Name,Review_Count,Rating,Price_Rank
7,BUQQF2QUuZXa6E3q3KzBxw,Twins Asian Bistro,31,4.5,NaN
9,qn_xkreIKL5RrdypG1yArg,Teriyaki Bowl,97,4.5,$
13,6AB55kxgUJTsURDezq3zCw,Best Teriyaki,141,4.5,$


In [183]:
tattoo_df.drop_duplicates(subset='Name', inplace=True)

In [184]:
print(tattoo_df['Name'].nunique())
print(tattoo_df.shape)

466
(466, 5)


## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [185]:
# write Pandas code to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

tattoo_id = tattoo_df['ID'].values

In [273]:
#tattoo_id

In [204]:
test_id = tattoo_id[0]
test_url =  'https://api.yelp.com/v3/businesses/{}/reviews'.format(test_id)
headers = {'Authorization': 'Bearer {}'.format(api_key)}

In [205]:
response = requests.get(test_url, headers=headers)

In [231]:
response.status_code

200

In [218]:
response.json() #['reviews'][0]['text']

{'possible_languages': ['en'],
 'reviews': [{'id': 'LULwvPoO2ljY9fXQYjB1Pg',
   'rating': 5,
   'text': "I've been wanting to get a tattoo for the longest time, and after doing some research, I came across Jing on Instagram and just knew I had to get my first...",
   'time_created': '2019-12-30 08:56:38',
   'url': 'https://www.yelp.com/biz/jings-tattoo-flushing?adjust_creative=9oud9pgPIhoXYt0000Ulyg&hrid=LULwvPoO2ljY9fXQYjB1Pg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=9oud9pgPIhoXYt0000Ulyg',
   'user': {'id': 'psSh0DDsrRfQWNL7eGHQMQ',
    'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/7UjJMP5gWMooKLAlr75mAg/o.jpg',
    'name': 'Monicka C.',
    'profile_url': 'https://www.yelp.com/user_details?userid=psSh0DDsrRfQWNL7eGHQMQ'}},
  {'id': 'VMKfqRBsJwQY93sTe3jcFg',
   'rating': 5,
   'text': "I came across a flash tattoo on Instagram and I instantly fell in love with Joseph's design. I loved the minimalistic and fine line style of his work and...",
   '

In [229]:
#yelp_call_rev(tattoo_id[4], api_key)

In [291]:
# write a function that takes a business id 
# and makes a call to the API for reivews

#tat_id is essentially tattoo_id[index number]

def yelp_call_rev(tat_id, api_key):
    url =  'https://api.yelp.com/v3/businesses/{}/reviews'.format(tat_id)
    
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    
    data = requests.get(url, headers=headers)
    
    response = data.json()
#     print(response.status_code)
    return response

In [294]:
dummy = yelp_call_rev(tattoo_id[4], api_key)

In [290]:
dummy.keys()

dict_keys(['reviews', 'total', 'possible_languages'])

In [279]:
dummy['reviews']

[{'id': 'B5fc58JbHI4L3wncC7Dzhg',
  'rating': 5,
  'text': 'Reached out last minute for a semi-impromptu tattoo. \n\nSu was amazing and discussed his worries about the placement and size I was thinking. \n\nTattoo itself...',
  'time_created': '2019-01-07 21:00:42',
  'url': 'https://www.yelp.com/biz/soma-art-tattoo-astoria?adjust_creative=9oud9pgPIhoXYt0000Ulyg&hrid=B5fc58JbHI4L3wncC7Dzhg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=9oud9pgPIhoXYt0000Ulyg',
  'user': {'id': 'SD9SQdL2xaheIE4y_F9fAA',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/PaiQhEwx8I0ekWnq6CzAUw/o.jpg',
   'name': 'Lauren F.',
   'profile_url': 'https://www.yelp.com/user_details?userid=SD9SQdL2xaheIE4y_F9fAA'}},
 {'id': 'R8OE2rwvGmuncqcxZdRHTA',
  'rating': 5,
  'text': "Su is AMAZING!!!!! The art, the price, everything is awesome! I highly recommend this artist for everybody who wants to add artistry on their bodies. can't...",
  'time_created': '2019-07-04 15:51:30',
  'url':

In [280]:
response.json()['reviews'][0]['text']

"I've been wanting to get a tattoo for the longest time, and after doing some research, I came across Jing on Instagram and just knew I had to get my first..."

In [260]:
response.json().keys()

dict_keys(['reviews', 'total', 'possible_languages'])

In [292]:
# Write a function to parse out the relevant information from the reviews

def parse_results_rev(results, tat_id):
    parsed_data = []
    
    for review in results:
        if 'reviews' in results.keys():
            for item in results['reviews']:
                rev_list = [
                            item['user']['name'],
                            item['text'],
                            item['rating'],
                            item['time_created'],
                            str(tat_id)
                            ]

                parsed_data.append(rev_list)
   
    return parsed_data

In [296]:
test_parse = parse_results_rev(dummy, tattoo_id[4])
test_parse

[['Lauren F.',
  'Reached out last minute for a semi-impromptu tattoo. \n\nSu was amazing and discussed his worries about the placement and size I was thinking. \n\nTattoo itself...',
  5,
  '2019-01-07 21:00:42',
  'K_EujGkKg0Pcv9lyKxVG0A'],
 ['Epirose B.',
  "Su is AMAZING!!!!! The art, the price, everything is awesome! I highly recommend this artist for everybody who wants to add artistry on their bodies. can't...",
  5,
  '2019-07-04 15:51:30',
  'K_EujGkKg0Pcv9lyKxVG0A'],
 ['Evelyn F.',
  'Su tattoed both my friend and me for our matching tattoos. From when we first came in, he was very kind to both of us and made us feel very comfortable. He...',
  5,
  '2019-04-29 12:26:52',
  'K_EujGkKg0Pcv9lyKxVG0A'],
 ['Lauren F.',
  'Reached out last minute for a semi-impromptu tattoo. \n\nSu was amazing and discussed his worries about the placement and size I was thinking. \n\nTattoo itself...',
  5,
  '2019-01-07 21:00:42',
  'K_EujGkKg0Pcv9lyKxVG0A'],
 ['Epirose B.',
  "Su is AMAZING!!!!!

In [311]:
columns_rev = ['Reviewer_Name', 'Review', 'Rating', 'Time','Tat_ID']
nothing_rev = pd.DataFrame(columns=columns_rev)
nothing_rev.to_csv('tatrev_api.csv')

In [293]:
# write a function to insert the parsed data into the reviews table

def data_save_rev(parsed_results_rev, csv_filename):
    
    columns_rev = ['Reviewer_Name', 'Review', 'Rating', 'Time', 'Tat_ID']
    
    update = pd.read_csv(csv_filename, index_col=0)
    
    former = pd.DataFrame(parsed_results_rev, columns=columns_rev)
    
    df_list_rev = [update, former]
    
    mega = pd.concat(df_list_rev)
    
    return mega.to_csv(csv_filename) 


In [304]:
data_save_rev(test_parse, 'rav_api.csv')

In [305]:
go = pd.read_csv('rav_api.csv', index_col=0)

go.drop_duplicates(subset='Review', inplace=True)
#go['Review'].nunique()

In [306]:
go

,Rating,Review,Reviewer Name,Reviewer_Name,Tat_ID,Time
0,5.0,Reached out last minute for a semi-impromptu t...,NaN,Lauren F.,K_EujGkKg0Pcv9lyKxVG0A,2019-01-07 21:00:42
1,5.0,"Su is AMAZING!!!!! The art, the price, everyth...",NaN,Epirose B.,K_EujGkKg0Pcv9lyKxVG0A,2019-07-04 15:51:30
2,5.0,Su tattoed both my friend and me for our match...,NaN,Evelyn F.,K_EujGkKg0Pcv9lyKxVG0A,2019-04-29 12:26:52


In [265]:
len(tattoo_id)

466

In [312]:
# combine the functions above into a single script




for tat_id in tattoo_id:

    first_call = yelp_call_rev(tat_id, api_key)
    second_call = parse_results_rev(first_call, tat_id)
    data_save_rev(second_call, 'tatrev_api.csv')

      
    


In [313]:
review_df = pd.read_csv('tatrev_api.csv', index_col=0)

In [316]:
review_df.drop_duplicates(subset='Review', inplace=True)

In [319]:
review_df.rename(columns={'Tat_ID':'ID'}, inplace=True)

In [321]:
review_df.head()

,Reviewer_Name,Review,Rating,Time,ID
0,Monicka C.,I've been wanting to get a tattoo for the long...,5.0,2019-12-30 08:56:38,2krcy6ooduY4QCHnRAzM-w
1,Michael Z.,I came across a flash tattoo on Instagram and ...,5.0,2019-10-03 15:28:40,2krcy6ooduY4QCHnRAzM-w
2,Lina T.,I reached out to Jing months in advance to boo...,5.0,2019-09-16 18:47:54,2krcy6ooduY4QCHnRAzM-w
0,Jamal K.,Took my girlfriend to get two pieces during th...,5.0,2020-02-26 15:59:24,amEkAWWagKNlkFuXIcOH5A
1,Andrea G.,Came here a couple weeks ago for a consultatio...,5.0,2019-12-03 03:29:40,amEkAWWagKNlkFuXIcOH5A


In [322]:
tattoo_df.head()

,ID,Name,Review_Count,Rating,Price_Rank
0,2krcy6ooduY4QCHnRAzM-w,Jing's Tattoo,80.0,4.5,$$
1,amEkAWWagKNlkFuXIcOH5A,Crush Ink,66.0,5.0,$$
2,HRI3N2_3Jy4YxO3fJ64OXA,Ching Tattoo,28.0,4.5,$$
3,At4uTuPCpZdB7J1u5N5LfQ,10 Thousand Foxes Tattoo,71.0,5.0,$$
4,K_EujGkKg0Pcv9lyKxVG0A,Soma Art Tattoo,34.0,4.5,$


In [345]:
#join business and review dataframes

df_ultra = pd.merge(tattoo_df, review_df, on='ID')

In [346]:
df_ultra.reset_index(inplace=True)
df_ultra.drop(labels='index', axis=1, inplace=True)

In [362]:
df_ultra.rename(columns={'Rating_x': 'Overall_Rating', 'Rating_y':'User_Rating'}, inplace=True)

In [363]:
df_ultra.head()

,ID,Name,Review_Count,Overall_Rating,Price_Rank,Reviewer_Name,Review,User_Rating,Time
0,2krcy6ooduY4QCHnRAzM-w,Jing's Tattoo,80.0,4.5,$$,Monicka C.,"I've been wanting to get a tattoo for the longest time, and after doing some research, I came across Jing on Instagram and just knew I had to get my first...",5.0,2019-12-30 08:56:38
1,2krcy6ooduY4QCHnRAzM-w,Jing's Tattoo,80.0,4.5,$$,Michael Z.,I came across a flash tattoo on Instagram and I instantly fell in love with Joseph's design. I loved the minimalistic and fine line style of his work and...,5.0,2019-10-03 15:28:40
2,2krcy6ooduY4QCHnRAzM-w,Jing's Tattoo,80.0,4.5,$$,Lina T.,I reached out to Jing months in advance to book an appointment for this past weekend. I wanted a geometric design of a Chihuahua in memory of my first dog....,5.0,2019-09-16 18:47:54
3,amEkAWWagKNlkFuXIcOH5A,Crush Ink,66.0,5.0,$$,Jamal K.,"Took my girlfriend to get two pieces during their flash sale. I inquired with them two days ago, not only did Shane make it super easy and was able to get...",5.0,2020-02-26 15:59:24
4,amEkAWWagKNlkFuXIcOH5A,Crush Ink,66.0,5.0,$$,Andrea G.,Came here a couple weeks ago for a consultation on getting a cover up. The two girls were very helpful with suggestions on the placement of the new tattoo I...,5.0,2019-12-03 03:29:40


## Part 5 -  Write Pandas code that will answer the questions posed. 

Now that your data is saved in CSVs, you can answer the questions. 

 
## Data Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


In [191]:
#Which are the 5 most reviewed businesses?
most_rev = tattoo_df.sort_values('Review_Count', ascending=False).head(5)
most_rev

,ID,Name,Review_Count,Rating,Price_Rank
41,rvNq0cz8zqAmCj3UZHyiQA,New York Adorned,465.0,4.5,$$$
34,NR24uDrPm7EsEHBtHT-F_Q,Village Tattoo NYC,406.0,4.5,$$
27,ztP04Diw9WQz_vDv52QLAg,Three Kings Tattoo,399.0,4.5,$$
22,1gZNMIAhd3qXMVGzkX5kuA,The End Is Near,354.0,4.5,$$
30,1472QFxEHKrkhDUfHYod6Q,Maria Tash,352.0,4.0,$$$


In [192]:
#What is the highest rating recieved in your data set and how many businesses have that rating?
tattoo_df.groupby('Rating').count().sort_values('Review_Count', ascending=False).head(1)

,ID,Name,Review_Count,Price_Rank
Rating,,,,
5.0,161,161,161,79


In [193]:
#What percentage of businesses have a rating greater than or 4.5?
print(round(tattoo_df[tattoo_df['Rating'] >= 4.5]['Name'].count()/len(tattoo_df) * 100), '%')

62.0 %


In [194]:
#What percentage of businesses have a rating less than 3?
print(round(tattoo_df[tattoo_df['Rating'] < 3]['Name'].count()/len(tattoo_df) * 100), '%')

9.0 %


In [195]:
#What is the average rating of restaurants that have a price label of one dollar sign? 
#Two dollar signs? Three dollar signs? 
tatoo_df.groupby('Price_Rank')['Rating'].mean()

Price_Rank
$       4.156250
$$      4.270833
$$$     3.928571
$$$$    3.142857
Name: Rating, dtype: float64

In [353]:
pd.set_option('display.max_colwidth', -1)

In [350]:
#Return the text of the reviews for the most reviewed restaurant.
df_ultra.sort_values(by='Review_Count', ascending=False).head(1)['Review']

118    I visited this location because the window full of dried flowers to be sold which I still have not found anywhere else in NY by the way. But to my surprise...
Name: Review, dtype: object

In [356]:
#Return the name of the business with the most recent review. 
df_ultra.sort_values(by='Time', ascending=False).head(1)['Name']

77    Three Kings Tattoo
Name: Name, dtype: object

In [364]:
conditions = df_ultra['Overall_Rating'] >= 5.0

In [365]:
#Find the highest rated business and return text of the most recent review. 
#If multiple business have the same rating, select the restaurant with the most reviews.

df_ultra[conditions].sort_values(by='Time', ascending=False).head(1)['Review']

278    I got a tattoo from Hannah a few weeks ago and I am sooo happy with it. I had an idea of what I wanted and did my best to describe it, and her drawing was...
Name: Review, dtype: object

In [367]:
#df_ultra['Overall_Rating'].unique()

In [368]:
second = df_ultra['Overall_Rating'] == 1 

In [369]:
#Find the lowest rated business and return text of the most recent review. 
#If multiple business have the same rating, select the restaurant with the least reviews. 
df_ultra[second].sort_values(by='Time', ascending=False).head(1)['Review']

1084    Artwork is dope ... can't say the same for his personality . Thinks he's a "funny guy". Doesn't work with his customers on their preference. Wouldn't recommend.
Name: Review, dtype: object

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    data_save(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50